##  Install & Imports


In [16]:
! pip install pandas sqlalchemy requests python-dotenv

In [17]:
import os
from datetime import datetime

import pandas as pd
from sqlalchemy import inspect

# import everything from your utils module:
from bitcoin_analytics_utils import (
    fetch_full_historical,
    fetch_realtime_df,
    ensure_table_exists,
    prune_old_rows,
    engine,
    TABLE_NAME,
)


## 2. Fetch Full Historical Data (Last 7 Days)

In [18]:
#  Fetch the last 7 days (7×24×60 minutes) of BTC/USD minute bars:
HISTORICAL_MINUTES = 7 * 24 * 60
df_hist = fetch_full_historical(HISTORICAL_MINUTES)

# Show first & last rows
print("Rows fetched:", len(df_hist))
display(df_hist.head())
display(df_hist.tail())
display(df_hist[["price_usd", "volume_usd", "volume_btc"]].describe())

Rows fetched: 10080


,timestamp,price_usd,volume_usd,volume_btc
0,2025-05-05 16:54:00,94268.55,813112.79,8.632
1,2025-05-05 16:55:00,94295.36,513676.01,5.450
2,2025-05-05 16:56:00,94261.91,1204569.26,12.790
3,2025-05-05 16:57:00,94254.45,597140.43,6.339
4,2025-05-05 16:58:00,94258.50,310937.76,3.300


,timestamp,price_usd,volume_usd,volume_btc
10075,2025-05-12 16:49:00,103044.74,1606969.08,15.60
10076,2025-05-12 16:50:00,103028.63,1276777.27,12.39
10077,2025-05-12 16:51:00,103013.95,1330898.41,12.92
10078,2025-05-12 16:52:00,102989.71,1648186.57,16.00
10079,2025-05-12 16:53:00,102967.90,0.00,0.00


,price_usd,volume_usd,volume_btc
count,10080.000000,1.008000e+04,10080.000000
mean,100477.564617,2.750719e+06,27.019899
std,3776.265293,2.891908e+07,279.425372
min,93411.390000,0.000000e+00,0.000000
25%,96871.767500,3.892748e+05,3.871000
50%,102812.075000,8.215775e+05,8.172000
75%,103725.797500,1.725621e+06,17.252500
max,105671.920000,1.555539e+09,15051.190000


## 3. Fetch Real‐Time Data (Last 24 Hours)

In [19]:
df_rt = fetch_realtime_df()
print("Realtime bars fetched:", len(df_rt))
display(df_rt.head())

Realtime bars fetched: 1441


,timestamp,price_usd,volume_usd,volume_btc
0,2025-05-11 16:54:00,103935.84,357259.92,3.437
1,2025-05-11 16:55:00,103950.20,216139.77,2.080
2,2025-05-11 16:56:00,103957.60,440408.61,4.236
3,2025-05-11 16:57:00,103988.83,381769.98,3.672
4,2025-05-11 16:58:00,103989.51,321841.99,3.095


## 4. Initialize / Ensure Database Table Exists

In [20]:
insp = inspect(engine)
if not insp.has_table(TABLE_NAME):
    print(f"→ Table `{TABLE_NAME}` missing; inserting historical data…")
    # This calls fetch_full_historical again under the hood, or reuse df_hist:
    df_hist.to_sql(TABLE_NAME, engine, if_exists="replace", index=False)
    print(f"  Inserted {len(df_hist)} rows.")
else:
    print(f"→ Table `{TABLE_NAME}` already exists; skipping initial load.")

→ Table `btc_minute_data123` already exists; skipping initial load.


## 5. Append New Real-Time Bars to DB


In [21]:
# 1. Get the max timestamp currently in the table
query = f"SELECT MAX(timestamp) AS max_ts FROM {TABLE_NAME}"
max_ts = pd.read_sql(query, engine)["max_ts"].iloc[0]
print("Latest timestamp in DB:", max_ts)

# 2. Filter `df_rt` for only truly new bars
new_bars = df_rt[df_rt["timestamp"] > max_ts]
print("New bars to append:", len(new_bars))

# 3. Append & prune if needed
if not new_bars.empty:
    with engine.begin() as conn:
        new_bars.to_sql(TABLE_NAME, conn, if_exists="append", index=False)
        prune_old_rows(conn)

Latest timestamp in DB: 2025-05-12 06:13:00
New bars to append: 641
  🔪 Pruned 641 old rows; now 2000 rows remain.


## 6. Verify Database Contents


In [22]:
# Show the 5 most recent records from the DB
df_db = pd.read_sql(
    f"SELECT timestamp, price_usd, volume_usd, volume_btc "
    f"FROM {TABLE_NAME} "
    f"ORDER BY timestamp DESC "
    f"LIMIT 5",
    engine,
)
display(df_db)

,timestamp,price_usd,volume_usd,volume_btc
0,2025-05-12 16:54:00,102968.68,0.00,0.00
1,2025-05-12 16:53:00,102969.25,1818349.06,17.66
2,2025-05-12 16:52:00,102989.71,1648186.57,16.00
3,2025-05-12 16:51:00,103013.95,1330898.41,12.92
4,2025-05-12 16:50:00,103028.63,1276777.27,12.39


In [23]:
print("Fetching real-time minute bars via wrapper...")
df_realtime = fetch_realtime_minute(API_KEY, period_minutes=10)
print(df_realtime.tail())D

Fetching real-time minute bars via wrapper...


NameError: name 'fetch_realtime_minute' is not defined